In [125]:
import sys
!{sys.executable} -m pip install -q langchain_core langchain-openai requests

In [126]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
import requests

In [127]:
# tool create
@tool
def multiply_two_numbers(a: int, b: int) -> int:
    """Multiply two numbers and return its value from third number """
    return a * b

In [128]:
print(multiply_two_numbers.invoke({"a": 5, "b": -6}))

-30


In [129]:
multiply_two_numbers.name

'multiply_two_numbers'

In [130]:
multiply_two_numbers.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [131]:
multiply_two_numbers.args_schema

langchain_core.utils.pydantic.multiply_two_numbers

## Tool Binding

Its is step where we registers **Custom** or **Structured** tools with LLM so that
a. LLM knows what all tools are available for its `functioning`.
b. LLM knows what each tools `does`.
c. LLM knows what `input format` to use for each tools. This is done via reading the schema of the tools.

In [132]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k",temperature=0, api_key="")

In [133]:
llm_with_tools = llm.bind_tools([multiply_two_numbers])

# Tool Calling

It is the process whre the LLM decides, during a conversation or task, that it needs to use a specific tool(function)
and generates a structured output(Schema) which has following: </br>

a. the `name of the tool`
b. and the `arguments` to call it with

The LLM does not run the tool, it just suggest the tool and the input argument. The actual execution is handled by **LanChain**.

In [134]:
message_list = []
message1 = HumanMessage(content="Hi How are you")
message_list.append(message1)
message_list.append(llm_with_tools.invoke ([message1]))

In [135]:
message_list

[HumanMessage(content='Hi How are you', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here to help you with any questions or tasks you have. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 58, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CgW4m1AauivaIxZ0tVyfrkVn1nU5V', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--90003d6f-4e50-4222-b3b5-efaf4e065598-0', usage_metadata={'input_tokens': 58, 'output_tokens': 37, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_

In [136]:
message2 = HumanMessage('Multiply two numbers where number 1 has value 9 and number 2 has value -18')
message_list.append(message2)
message_list

[HumanMessage(content='Hi How are you', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here to help you with any questions or tasks you have. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 58, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CgW4m1AauivaIxZ0tVyfrkVn1nU5V', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--90003d6f-4e50-4222-b3b5-efaf4e065598-0', usage_metadata={'input_tokens': 58, 'output_tokens': 37, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_

In [137]:
result = llm_with_tools.invoke(message_list)
message_list.append(result)
message_list

[HumanMessage(content='Hi How are you', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here to help you with any questions or tasks you have. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 58, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CgW4m1AauivaIxZ0tVyfrkVn1nU5V', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--90003d6f-4e50-4222-b3b5-efaf4e065598-0', usage_metadata={'input_tokens': 58, 'output_tokens': 37, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_

When message1 is 'Hi How are you',
* then AI message content is *AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here to help you with any questions or tasks you have. How can I assist you today?",*

When message2 is ' Multiplication of two numbers'
* then AI message content is spaces  AIMessage(content='',
* and it has tool_calls which means it does not uses its own context but fetches the tools it was binded with
  *tool_calls=[{'name': 'multiply_two_numbers', 'args': {'a': 9, 'b': -18, 'c': 0}, 'id': 'call_w95szkN0ZTGPYQX6tfXh4jXy', 'type': 'tool_call'}]*




## Tool Execution

It is the step where the actual Tool is run using the input arguments that LLM suggested during tool calling.

eg.
LLM says 'Hey, call the multiply tool with input parameters a =10, b= 20  </br>
Tool Execution is when LangChain actually run the mutiply operation (a =10, b= 20 ) and get the result = 200

In [140]:
args= result.tool_calls[0]
args

{'name': 'multiply_two_numbers',
 'args': {'a': 9, 'b': -18},
 'id': 'call_khwhUGcguRXvI7lvsDOunz6S',
 'type': 'tool_call'}

In [142]:
tool_result = multiply_two_numbers.invoke(args)
tool_result

ToolMessage(content='-162', name='multiply_two_numbers', tool_call_id='call_khwhUGcguRXvI7lvsDOunz6S')

In [143]:
message_list.append(tool_result)
message_list

[HumanMessage(content='Hi How are you', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here to help you with any questions or tasks you have. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 58, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CgW4m1AauivaIxZ0tVyfrkVn1nU5V', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--90003d6f-4e50-4222-b3b5-efaf4e065598-0', usage_metadata={'input_tokens': 58, 'output_tokens': 37, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_

In [144]:
llm_with_tools.invoke (message_list).content

'The result of multiplying 9 and -18 is -162.'